# Analyse et Modélisation SARIMAX
## Prédiction des Admissions - Vision 2026

Ce notebook implémente un modèle **SARIMAX** (Seasonal AutoRegressive Integrated Moving Average with eXogenous variables) pour capturer la saisonnalité et les facteurs externes (jours fériés, week-ends) influençant les admissions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

### 1. Préparation des Données

In [ ]:
# Chargement des données
df = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df['date_entree'] = pd.to_datetime(df['date_entree'])

# Agrégation journalière
y = df.groupby('date_entree').size().rename('admissions').asfreq('D', fill_value=0)

print(f"Période : {y.index.min()} à {y.index.max()}")
print(f"Moyenne d'admissions : {y.mean():.2f}/jour")

### 2. Analyse Statistique & Décomposition
Le modèle SARIMAX nécessite de comprendre la saisonnalité (hebdomadaire, mensuelle).

In [ ]:
# Décomposition saisonnière (Période 7 pour l'hebdomadaire)
result = seasonal_decompose(y, model='additive', period=7)
fig = result.plot()
fig.set_size_inches(12, 8)
plt.show()

### 3. Variables Exogènes
Nous ajoutons les week-ends et jours fériés comme facteurs d'influence.

In [ ]:
exog = pd.DataFrame(index=y.index)
exog['is_weekend'] = (exog.index.dayofweek >= 5).astype(int)

holidays = ['2024-01-01', '2024-04-01', '2024-05-01', '2024-05-08', 
            '2024-05-09', '2024-05-20', '2024-07-14', '2024-08-15', 
            '2024-11-01', '2024-11-11', '2024-12-25']
exog['is_holiday'] = exog.index.strftime('%Y-%m-%d').isin(holidays).astype(int)

print(exog.head())

### 4. Entraînement du Modèle SARIMAX
Nous utilisons une configuration (1, 1, 1) x (1, 1, 1, 7) comme point de départ.

In [ ]:
# Split Train/Test
train_size = int(len(y) * 0.9)
train, test = y.iloc[:train_size], y.iloc[train_size:]
exog_train, exog_test = exog.iloc[:train_size], exog.iloc[train_size:]

# Modèle SARIMAX
model = SARIMAX(train, 
                exog=exog_train,
                order=(1, 1, 1), 
                seasonal_order=(1, 1, 1, 7), 
                enforce_stationarity=False, 
                enforce_invertibility=False)

model_fit = model.fit(disp=False)
print(model_fit.summary())

### 5. Prédictions & Évaluation

In [ ]:
predictions = model_fit.forecast(steps=len(test), exog=exog_test)
mae = mean_absolute_error(test, predictions)
rmse = np.sqrt(mean_squared_error(test, predictions))

print(f"MAE : {mae:.2f}")
print(f"RMSE : {rmse:.2f}")

# Graphique
plt.figure(figsize=(15, 6))
plt.plot(train.tail(60), label='Train (Derniers 60j)', color='lightgrey')
plt.plot(test, label='Réel (Test)', color='blue')
plt.plot(test.index, predictions, label='SARIMAX Predictions', color='red', linestyle='--')
plt.title('SARIMAX : Prédictions vs Réalité')
plt.legend()
plt.show()